# issue
* tokenize 작업의 속도가 굉장히 느림 개선방법이 있다면 고민해봐야함 (작업내용 1차적으로 보존)

In [2]:
# import
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm,tqdm_notebook  # 진행과정 시각화
tqdm.pandas() #apply사용
from datetime import timedelta  # 시간날짜
from apyori import apriori  # 연관분석

import re
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import gc
import os

import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup  # 크롤링

import wordcloud
from PIL import Image
import random

import konlpy
from konlpy.tag import Komoran,Okt,Kkma,Twitter  # 자연어처리
# from eunjeon import Mecab #은전한닢
okt = Okt(max_heap_size=5120)
# mecab = Mecab()
komoran = Komoran(userdic='user_dictionary.txt')

#한글깨짐방지
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

C:\Users\User\Anaconda3\lib\site-packages\jpype\_core.py:217: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [2]:
def tokenized(dataframe, file_name, keyword, stopword):
    '''
    okt : 형태소분석기 무엇사용?
    dataframe : 분석할 df
    keyword : sw에 추가하기 위함
    '''

    # stopword에 검색어 추가
    stopword.append(keyword.split(' ')[0])
    
    start = 0
    end = len(dataframe)
    
    print(keyword)
    if end<=20000:    
        # tqdm.pandas(desc = '토큰화')
        token_df = dataframe.iloc[start:end]['full_text'].progress_apply(lambda x: okt.pos(x, norm=True, stem=True))
        token_df_e = token_df.explode()
        # tqdm.pandas(desc = '단어추출')
        token_noun = token_df_e.apply(lambda x: x[0] if x[1] == 'Noun' else np.nan).dropna()
        token_adj = token_df_e.apply(lambda x: x[0] if x[1] == 'Adjective' else np.nan).dropna()

        # 클리닝
        # tqdm.pandas(desc = '클리닝')
        token_noun = token_noun[token_noun.apply(lambda word: (len(word) > 1) & (word not in sw))]
        token_adj = token_adj[token_adj.apply(lambda word: (len(word) > 1) & (word not in sw))]
        
        #저장
        concated_df = pd.concat([token_noun,token_adj],axis = 1)
        concated_df.to_csv('D:/Python/블로그크롤링/output/token/{keyword}_{start}~{end}.csv'.format(keyword = keyword, start = start, end = end))
#         # 저장
# #         defalut = pd.Series(index=range(start, end), data=range(start, end), name='index')
#         dataframe = pd.merge(dataframe, token_noun.groupby(level=0).agg(list).rename(
#             'Noun'), how='left', left_index=True, right_index=True)
#         dataframe = pd.merge(dataframe, token_adj.groupby(level=0).agg(list).rename(
#             'Adjective'), how='left', left_index=True, right_index=True)
#         pd.DataFrame(dataframe).to_csv('D:/Python/블로그크롤링/output/token/{keyword}_{start}~{end}.csv'.format(keyword = keyword, start = start, end = end))
        
    else : #20000개 이상인경우 분할 작업
        final_end = len(dataframe)
        epoch = (final_end//20000) +1
        start = 0
        end = 20000
        while epoch != 0:
            # tqdm.pandas(desc = '토큰화')
            token_df = dataframe.iloc[start:end]['full_text'].progress_apply(lambda x: okt.pos(x, norm=True, stem=True))
            token_df_e = token_df.explode()
            # tqdm.pandas(desc = '단어추출')
            token_noun = token_df_e.apply(lambda x: x[0] if x[1] == 'Noun' else np.nan).dropna()
            token_adj = token_df_e.apply(lambda x: x[0] if x[1] == 'Adjective' else np.nan).dropna()

            # 클리닝
            # tqdm.pandas(desc = '클리닝')
            token_noun = token_noun[token_noun.apply(lambda word: (len(word) > 1) & (word not in sw))]
            token_adj = token_adj[token_adj.apply(lambda word: (len(word) > 1) & (word not in sw))]

            #저장
            concated_df = pd.concat([token_noun,token_adj],axis = 1)
            os.makedirs(
            './output/token/{file_name}'.format(file_name=file_name), exist_ok=True)
            concated_df.to_csv('D:/Python/블로그크롤링/output/token/{keyword}_{start}~{end}.csv'.format(keyword = keyword, start = start, end = end))
            
#             # defalut = pd.Series(index=range(start, end), data=range(start, end), name='index')
#             dataframe = pd.merge(dataframe, token_noun.groupby(level=0).agg(list).rename(
#                 'Noun'), how='left', left_index=True, right_index=True)
#             dataframe = pd.merge(dataframe, token_adj.groupby(level=0).agg(list).rename(
#                 'Adjective'), how='left', left_index=True, right_index=True)
            
#             os.makedirs(
#             './output/token/{file_name}'.format(file_name=file_name), exist_ok=True)
#             pd.DataFrame(dataframe).to_csv('D:/Python/블로그크롤링/output/token/{file_name}/{file_name}_{start}~{end}.csv'.format(file_name = file_name, start = start, end = end))
            
            #next turn
            if epoch >2 :
                start += 20000
                end += 20000
            else :      
                start += 20000
                end = final_end

            tokens_dict = dict()
            tokens_dict['Nouns'] = []
            tokens_dict['Adj'] = []
            epoch -= 1

In [3]:
# sw = list(pd.read_csv("stopword.csv")['불용어']) #불용어 불러오기
# path = 'D:/Python/블로그크롤링/output/크롤링/통합/'
# file_list = os.listdir(path)
# for file in file_list:
#     main_df = pd.read_csv(path + file)
#     keyword = file_list[0].split('_')[0]
#     tokenized(okt,main_df,keyword,sw)
    

In [7]:
sw = list(pd.read_csv("stopword.csv")['불용어']) #불용어 불러오기
path = 'D:/Python/블로그크롤링/output/크롤링/통합/'
file_list = os.listdir(path)
file = '광진 +양양_통합_5044.csv'
main_df = pd.read_csv(path + file)
keyword = '양양'

### 방법2
tokenize을 일부부분을 apply로 진행, 10000개 10분, 메모리 에러문제로 그이상 확인불가  
tqdm.pandas()로 apply 진행상황확인

In [25]:
main_df.shape[0]

85670

In [30]:
# 저장할 위치
tokens_dict = dict()
tokens_dict['Nouns'] = []
tokens_dict['Adj'] = []

start = 70000
end = 85670
# tqdm.pandas(desc = '토큰화')
a = main_df.iloc[start:end]['full_text'].progress_apply(lambda x: okt.pos(x, norm=True, stem=True))
a_e = a.explode()
# tqdm.pandas(desc = '단어추출')
a_e_n = a_e.apply(lambda x: x[0] if x[1] == 'Noun' else np.nan).dropna()
a_e_ad = a_e.apply(lambda x: x[0] if x[1] == 'Adjective' else np.nan).dropna()

# 클리닝
# tqdm.pandas(desc = '클리닝')
a_e_n = a_e_n[a_e_n.apply(lambda word: (len(word) > 1) & (word not in sw))]
a_e_ad = a_e_ad[a_e_ad.apply(lambda word: (len(word) > 1) & (word not in sw))]


# 저장
defalut = pd.Series(index=range(start, end), data=range(start, end), name='index')
defalut = pd.merge(defalut, a_e_n.groupby(level=0).agg(list).rename(
    'Noun'), how='left', left_index=True, right_index=True)
defalut = pd.merge(defalut, a_e_ad.groupby(level=0).agg(list).rename(
    'Adjective'), how='left', left_index=True, right_index=True)
pd.DataFrame(defalut).to_csv('D:/Python/블로그크롤링/output/token/강원도_{start}~{end}.csv'.format(start = start, end = end))

100%|████████████████████████████████████████████████████████████████████████████| 15670/15670 [18:21<00:00, 14.22it/s]


### 속도가 느려서 튜닝

In [8]:
#ori 
#저장할 위치
tokens_dict = dict()
tokens_dict['Nouns'] = []
tokens_dict['Adj'] = []
for blog_num in tqdm_notebook(range(0,len(main_df[:500])),desc = keyword+'tokenize'):
    #블로그 품사태깅 (load한 경우 list가 아닌 text로 받아서 첫번째가 아닌 전체를 받는다)
#     pos_text = okt.pos(target[blog_num][0], norm=True, stem=True)
        pos_text = okt.pos(main_df['full_text'][blog_num], norm=True, stem=True)

        pos_text_df = pd.DataFrame(pos_text)
        noun_lists = pos_text_df[pos_text_df[1].apply(lambda x : (x in "Noun"))][0].values
        adj_lists = pos_text_df[pos_text_df[1].apply(lambda x : (x in "Adjective"))][0].values
        #클리닝
        clean_nouns = list(filter(lambda word : (len(word)>1)&(word not in sw),noun_lists))
        clean_adjs = list(filter(lambda word : (len(word)>1)&(word not in sw),adj_lists))
        #dictionay 저장
        tokens_dict.get('Nouns',0).append(clean_nouns)
        tokens_dict.get('Adj',0).append(clean_adjs)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
#tunning 1
#저장할 위치
tokens_dict = dict()
tokens_dict['Nouns'] = []
tokens_dict['Adj'] = []

for txt in tqdm_notebook(main_df['full_text'].values[:500] ,desc = keyword+'tokenize'):
    a = okt.pos(txt, norm=True, stem=True)
    noun_lists = []
    adj_lists = []
    for word, dtype in a:
        if dtype == 'Noun':
            noun_lists.append(word)
        elif dtype == 'Adjective':
            adj_lists.append(word)
    #클리닝
    clean_nouns = list(filter(lambda word : (len(word)>1)&(word not in sw),noun_lists))
    clean_adjs = list(filter(lambda word : (len(word)>1)&(word not in sw),adj_lists))
    #dictionay 저장
    tokens_dict.get('Nouns',0).append(clean_nouns)
    tokens_dict.get('Adj',0).append(clean_adjs)

In [ ]:
#tunning 2 : DF.isin
#저장할 위치
tokens_dict = dict()
tokens_dict['Nouns'] = []
tokens_dict['Adj'] = []
num = 1
count = 1
for txt in tqdm_notebook(main_df['full_text'].values[:500] ,desc = keyword+'tokenize'):
    
    a = okt.pos(txt, norm=True, stem=True)

    a_df = pd.DataFrame(a)
    noun_lists = a_df[a_df[1].isin(['Noun'])][0].values
    adj_lists = a_df[a_df[1].isin(['Adjective'])][0].values
    #클리닝
    clean_nouns = list(filter(lambda word : (len(word)>1)&(word not in sw),noun_lists))
    clean_adjs = list(filter(lambda word : (len(word)>1)&(word not in sw),adj_lists))
    #dictionay 저장
    tokens_dict.get('Nouns',0).append(clean_nouns)
    tokens_dict.get('Adj',0).append(clean_adjs)
    
    gc.collect()
    num += 1
    
    if num == 100:
        #중간저장 
        tokens_df = pd.DataFrame(tokens_dict)
        tokens_df.to_csv('D:/Python/블로그크롤링/output/token/{keyword}_token_{count}.csv'.format(keyword = keyword,count = count)) #token을 원래문장과 비교하려면 동일한 인덱스로 추정하면됨
        count += 1
        num = 1

In [ ]:
#tunning 3 : np.isin
#저장할 위치
tokens_dict = dict()
tokens_dict['Nouns'] = []
tokens_dict['Adj'] = []
#append 미리선언
add_noun = tokens_dict.get('Nouns',0).append
add_adj = tokens_dict.get('Adj',0).append
for txt in tqdm_notebook(main_df['full_text'].values[:500] ,desc = keyword+'tokenize'):
    tagged = okt.pos(txt, norm=True, stem=True)
    
    #array로 변환후 작업
    tagged_arr= np.array(tagged)

#     n_mask = np.isin(tagged_arr[:,1],"Noun")
#     a_mask = np.isin(tagged_arr[:,1],"Adjective")

    noun_lists = tagged_arr[np.isin(tagged_arr[:,1],"Noun")][:,0]
    adj_lists = tagged_arr[np.isin(tagged_arr[:,1],"Adjective")][:,0]
    #클리닝
    clean_nouns = list(filter(lambda word : (len(word)>1)&(word not in sw),noun_lists))
    clean_adjs = list(filter(lambda word : (len(word)>1)&(word not in sw),adj_lists))
    #dictionay 저장
    add_noun(clean_nouns)
    add_adj(clean_adjs)